# <center>Dunnhumby - The Complete Journey<center>

** ** 
    
# <center>*01 - Gold Layer*<center>
    

This notebook works as an extention to the work developed using *Databricks*. Here all datasets will be addressed and transformed accordingly.

<br>
    
This project was developed by <br><br>

*<center>António Oliveira | NTT Data Summer Internship 2024<center>*

** **

<a class="anchor" id="0"></a>

# Table of Contents

1. [Importing Libraries & Data](#1.-Importing-Libraries-&-Data)

    1.1 [Libraries](#1.1-Libraries)
    
    1.2 [Data](#1.2-Data) <br><br>
  
    
2. [HH Demographics Dataset](#2.-HH-Demographic)

    2.1 [Feature Engineering](#2.1-Feature-Engineering) <br><br>
    
3. [Transactions Dataset](#3.-Transaction)    <br>

    3.1 [Feature Engineering](#3.1-Feature-Engineering) <br>
    
    3.2 [Dataset Transformation](#3.2-Dataset-Transformation) <br><br>
    
4. [Campaigns Dataset](#4.-Campaign) <br>

    4.1 [Dataset Transformation](#4.1-Dataset-Transformation) <br><br>
    
5. [Coupons Dataset](#5.-Coupon) <br>

    5.1 [Dataset Transformation](#5.1-Dataset-Transformation) <br><br>
    
6. [Product Dataset](#6.-Product)

    6.1 [Feature Engineering](#6.1-Feature-Engineering) <br><br>
    
7. [Causal Dataset](#7.-Causal) 

    7.1 [Dataset Transformation](#7.1-Dataset-Transformation) <br><br>
    
8. [Export](#8.-Export) <br><br>

## 1. Importing Libraries & Data

### 1.1 Libraries

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

import functions

## 1.2 Data

In [3]:
#path = '/Users/antoniooliveira/Downloads/NTT project'
path = "C:/Users/aprataso/Downloads/final_data"

hh = pd.read_csv(f"{path}/hh.csv")
transaction = pd.read_csv(f"{path}/trans.csv")
campaign = pd.read_csv(f"{path}/camp.csv")
coupon = pd.read_csv(f"{path}/coupon.csv")
prod = pd.read_csv(f"{path}/prod.csv")
causal = pd.read_csv(f"{path}/causal.csv")



## 2. HH Demographic

<a class='anchor' id='1'></a>
[Top &#129033;](#0) 

In [5]:
hh.head(3)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,household_key,marital_status,age_group,adult_category_size,has_kids
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1,married,senior,2.0,unknown
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7,married,middle-aged,2.0,unknown
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8,unknown,adult,2.0,Yes


Using *info* to explore datatypes and missing values

In [6]:
hh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 801 entries, 0 to 800
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   age_desc             801 non-null    object 
 1   marital_status_code  801 non-null    object 
 2   income_desc          801 non-null    object 
 3   homeowner_desc       801 non-null    object 
 4   hh_comp_desc         801 non-null    object 
 5   household_size_desc  801 non-null    object 
 6   kid_category_desc    801 non-null    object 
 7   household_key        801 non-null    int64  
 8   marital_status       801 non-null    object 
 9   age_group            801 non-null    object 
 10  adult_category_size  728 non-null    float64
 11  has_kids             801 non-null    object 
dtypes: float64(1), int64(1), object(10)
memory usage: 75.2+ KB


### 2.1 Feature Engineering

**Has Kids**

Replace *Yes* and *No* in has kids by boolean

In [7]:
hh['has_kids'] = hh['has_kids'].replace({
    'Yes': 1,
    'No': 0,
    'unknown': np.nan
})

**Average Age**

In [8]:
hh['min_age'] = hh['age_desc'].str[:2]
hh['max_age'] = hh['age_desc'].apply(lambda x: x[-2:] if len(x) > 3 else 65)

hh['min_age'] = hh['min_age'].astype('int')
#hh['min_age'].unique()


array([65, 45, 25, 35, 19, 55])

In [9]:
hh['max_age'] = hh['max_age'].astype(int)
#hh['max_age'].unique()

array([65, 54, 34, 44, 24, 64])

Dropping Temporary Variables

In [10]:
hh['avg_age'] = (hh['min_age'] + hh['max_age']) / 2

hh = hh.drop(['min_age', 'max_age'], axis = 1)

**Average Income**

In [11]:
hh['income_'] = hh['income_desc'].str.findall(r'\d+')
hh['income_'] = hh['income_'].apply(lambda x: list(map(int, x)) if x else [])
hh[['income_', 'income_desc']]

,income_,income_desc
0,"[35, 49]",35-49K
1,"[50, 74]",50-74K
2,"[25, 34]",25-34K
3,"[75, 99]",75-99K
4,"[50, 74]",50-74K
...,...,...
796,"[50, 74]",50-74K
797,"[75, 99]",75-99K
798,"[35, 49]",35-49K
799,"[50, 74]",50-74K


In [12]:
hh['income_desc'].unique()

array(['35-49K', '50-74K', '25-34K', '75-99K', 'Under 15K', '100-124K',
       '15-24K', '125-149K', '150-174K', '250K+', '175-199K', '200-249K'],
      dtype=object)

In [13]:
hh[hh['income_desc'] == '200-249K']

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,household_key,marital_status,age_group,adult_category_size,has_kids,avg_age,income_
280,45-54,A,200-249K,Homeowner,2 Adults No Kids,2,None/Unknown,844,married,middle-aged,2.0,NaN,49.5,"[200, 249]"
293,35-44,A,200-249K,Homeowner,2 Adults Kids,3,1,871,married,middle-aged,2.0,1.0,39.5,"[200, 249]"
318,19-24,A,200-249K,Unknown,2 Adults No Kids,2,None/Unknown,941,married,young adult,2.0,NaN,21.5,"[200, 249]"
328,25-34,A,200-249K,Homeowner,2 Adults Kids,5+,3+,976,married,adult,2.0,1.0,29.5,"[200, 249]"
714,35-44,A,200-249K,Homeowner,2 Adults No Kids,2,None/Unknown,2231,married,middle-aged,2.0,NaN,39.5,"[200, 249]"


In [14]:
hh['avg_income'] = hh['income_'].apply(lambda x: sum(x) / len(x) * 1000 if len(x) > 0 else None)
hh['avg_income'] = hh['avg_income'].astype(int)
hh['avg_income']

hh = hh.drop(['income_'], axis = 1)

Addressing **Category Sizes** in *kid_category_desc* and *household_size_desc*

In [15]:
hh['n_kids'] = hh['kid_category_desc'].replace({
    '1': 1,
    '2': 2,
    '3+': 3,
    'None/Unknown': np.nan
})

hh['n_kids'] = hh['n_kids'].astype(float)

hh = hh.drop('kid_category_desc', axis = 1)

In [16]:
hh['n_household'] = hh['household_size_desc'].replace({
    '1': 1,
    '2': 2,
    '3': 3,
    '4': 4,
    '5+': 5
})

hh['n_household'] = hh['n_household'].astype(int)

hh = hh.drop('household_size_desc', axis = 1)

In [17]:
#hh['household_size_desc'].unique()

**Gender**

In [18]:
hh['hh_comp_desc'].unique()

array(['2 Adults No Kids', '2 Adults Kids', 'Single Female', 'Unknown',
       'Single Male', '1 Adult Kids'], dtype=object)

In [19]:
mapping = {
     'Single Male': 0,
     'Single Female': 1,
     '2 Adults No Kids': 2,
     '2 Adults Kids': 2
}

hh['gender(s)'] = hh['hh_comp_desc'].map(mapping)

Since *hh_comp_desc* has the value 1 Adult Kids it is impossible to retrieve its gender. With that in mind, next  the % of males and females already in the dataset (retrieved from values *Single Female* and *Single Male*) will be computed and used as a probability to fill the gender of the observations that correspond *hh_comp_desc = '1 adult kids'*

In [20]:
# Filter out rows where 'gender(s)' is not 0 or 1
filtered_hh = hh[hh['gender(s)'].isin([0, 1])]

# Calculate total number of rows after filtering
total_rows_filtered = len(filtered_hh)

# Count occurrences of each value in 'gender(s)' after filtering
gender_counts_filtered = filtered_hh['gender(s)'].value_counts()

# Percentages for 'Single Male' (0) and 'Single Female' (1)
male_count_filtered = gender_counts_filtered.get(0, 0)  # Count of males
female_count_filtered = gender_counts_filtered.get(1, 0)  # Count of females

male_probability = male_count_filtered / total_rows_filtered
female_probability = female_count_filtered / total_rows_filtered

# Filter for "1 Adult Kids"
adult_kids_filter = hh['hh_comp_desc'] == '1 Adult Kids'

# Randomly assign 'male' or 'female' based on calculated probabilities
np.random.seed(0)  # For reproducibility
assign_gender = np.random.choice([0, 1], size=adult_kids_filter.sum(), p=[male_probability, female_probability])

# Update the 'gender(s)' column with the assigned genders
hh.loc[adult_kids_filter, 'gender(s)'] = assign_gender

#hh = hh.drop('hh_comp_desc', axis = 1)

After addressing values with '1 Adult Kids' we must also address the observations where *hh_comp_desc = 'Unknown'*

In [21]:
hh[hh['hh_comp_desc'] == 'Unknown']#.head(3)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_key,marital_status,age_group,adult_category_size,has_kids,avg_age,avg_income,n_kids,n_household,gender(s)
10,35-44,U,50-74K,Unknown,Unknown,25,unknown,middle-aged,NaN,NaN,39.5,62000,NaN,1,NaN
51,45-54,U,50-74K,Unknown,Unknown,136,unknown,middle-aged,NaN,NaN,49.5,62000,NaN,1,NaN
56,45-54,U,50-74K,Homeowner,Unknown,158,unknown,middle-aged,NaN,NaN,49.5,62000,NaN,1,NaN
74,35-44,U,50-74K,Homeowner,Unknown,216,unknown,middle-aged,NaN,NaN,39.5,62000,NaN,1,NaN
78,65+,U,15-24K,Unknown,Unknown,221,unknown,senior,NaN,NaN,65.0,19500,NaN,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
753,55-64,U,100-124K,Homeowner,Unknown,2342,unknown,senior,NaN,NaN,59.5,112000,NaN,1,NaN
758,25-34,B,50-74K,Homeowner,Unknown,2360,single,adult,NaN,NaN,29.5,62000,NaN,1,NaN
766,45-54,U,50-74K,Homeowner,Unknown,2380,unknown,middle-aged,NaN,NaN,49.5,62000,NaN,1,NaN
782,45-54,U,35-49K,Unknown,Unknown,2445,unknown,middle-aged,NaN,NaN,49.5,42000,NaN,1,NaN


Above we can see that most observations where *hh_comp_desc = 'Unknown'* have missing values in other columns.

Consequently, after a careful analysis, a **series of conditions** can be created in order to address both these issues.

if household_size_desc = 1 then adult_category_size = 1

In [22]:
hh.loc[hh['n_household'] == 1, 'adult_category_size'] = 1


if adult_category_size = 1 then has_kids = 0 and n_kids = 0

In [23]:
hh.loc[hh['adult_category_size'] == 1, 'n_kids'] = 0
hh.loc[hh['adult_category_size'] == 1, 'has_kids'] = 0



if adult_category_size is null then adult_category_size= n_household - n_kids

In [24]:
hh.loc[hh['adult_category_size'].isna(), 'adult_category_size'] = (
    hh['n_household'] - hh['n_kids']
)

if adult_category_size = 2 and gender(s) is null -> gender(s) = 2

In [25]:
hh.loc[(hh['adult_category_size'] == 2) & (hh['gender(s)'].isna()), 'gender(s)'] = 2

if n_kids = 1 -> has kids = 1

In [26]:
hh.loc[hh['n_kids'] == 1, 'has_kids'] = 1

if n_household = 1 or = 2 and gender is null, assign gender given previously computed probabilties

In [27]:
filtered_hh = hh[(hh['n_household'] == 1) | (hh['n_household'] == 2) & (hh['gender(s)'].isna())]

assign_gender = np.random.choice([0, 1], size=len(filtered_hh), p=[male_probability, female_probability])

hh.loc[filtered_hh.index, 'gender(s)'] = assign_gender


if adult_category_size = 1 and marital_status = unknown --> marital_status = Single and marital_status_code = U

In [28]:
hh.loc[(hh['adult_category_size'] == 1) & (hh['marital_status'] == 'unknown'), 'marital_status'] = 'single'
hh.loc[(hh['adult_category_size'] == 1) & (hh['marital_status'] == 'single'), 'marital_status_code'] = 'U'

if adult_category_size = 2 and has_kids is null -> has_kids = False

In [29]:
hh.loc[(hh['adult_category_size'] == 2) & (hh['has_kids'].isna()), 'has_kids'] = 0


if adult_category_size = 2 and n_kids is null -> n_kids = 0

In [30]:
hh.loc[(hh['adult_category_size'] == 2) & (hh['n_kids'].isna()), 'n_kids'] = 0

if marital_status_code = A and hh_comp_desc = Unknown --> adult_category_size = 2 and has_kids = False and n_kids = 0 and gender(s) = 2

In [31]:
condition = (hh['marital_status_code'] == 'A') & (hh['hh_comp_desc'] == 'Unknown')

hh.loc[condition, 'adult_category_size'] = 2
hh.loc[condition, 'has_kids'] = 0
hh.loc[condition, 'n_kids'] = 0
hh.loc[condition, 'gender(s)'] = 2

Checking if all the missing values were treated

In [32]:
hh.loc[hh['adult_category_size'].isna()]

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_key,marital_status,age_group,adult_category_size,has_kids,avg_age,avg_income,n_kids,n_household,gender(s)


Drop replaced Variables

In [33]:
hh = hh.drop(['hh_comp_desc', 'income_desc', 'age_desc'], axis = 1)

## 3. Transaction

<a class='anchor' id='1'></a>
[Top &#129033;](#0) 

In [4]:
transaction.head(2)

,household_key,basket_id,day,product_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,sales_value_eu,loyalty_card_price,non_loyalty_card_price,days_,transaction_date
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0,1.49,1.99,1.39,1,2021-01-01
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0,0.88,0.82,0.82,1,2021-01-01


In [5]:
#transaction[transaction['quantity'] > 50000]

,household_key,basket_id,day,product_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,sales_value_eu,loyalty_card_price,non_loyalty_card_price,days_,transaction_date
166536,149,28210551971,103,6534178,51912,110.00,327,-5.19,1514,15,0.0,0.0,117.70,0.0,0.0,103,2021-04-13
468356,2407,29392047893,181,6544236,85055,210.00,375,-8.51,1606,27,0.0,0.0,224.70,0.0,0.0,181,2021-06-30
481876,630,29484790880,185,6534178,61335,150.21,384,-6.13,1056,27,0.0,0.0,160.72,0.0,0.0,185,2021-07-04
748962,630,34749153595,503,6534178,89638,250.00,384,-13.45,1927,73,0.0,0.0,267.50,0.0,0.0,503,2022-05-18


Using *info* to explore datatypes and missing values

In [35]:
transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2595732 entries, 0 to 2595731
Data columns (total 17 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   household_key           int64  
 1   basket_id               int64  
 2   day                     int64  
 3   product_id              int64  
 4   quantity                int64  
 5   sales_value             float64
 6   store_id                int64  
 7   retail_disc             float64
 8   trans_time              int64  
 9   week_no                 int64  
 10  coupon_disc             float64
 11  coupon_match_disc       float64
 12  sales_value_eu          float64
 13  loyalty_card_price      float64
 14  non_loyalty_card_price  float64
 15  days_                   int64  
 16  transaction_date        object 
dtypes: float64(7), int64(9), object(1)
memory usage: 336.7+ MB


### 3.1 Feature Engineering

Transform Prices in Euros

In [36]:
transaction['loyalty_eu'] = transaction['loyalty_card_price'] * 1.07
transaction['non_loyalty_eu'] = transaction['non_loyalty_card_price'] * 1.07

transaction['loyalty_eu'] = transaction['loyalty_eu'].round(2)
transaction['non_loyalty_eu'] = transaction['non_loyalty_eu'].round(2)

In [37]:
transaction = transaction.drop(['days_', 'loyalty_card_price', 'non_loyalty_card_price', 'sales_value'], axis = 1)

### 3.2 Dataset Transformation

Create a new Dataframe basket_id where each line is grouped by basket_id

In [38]:
basket = transaction.groupby(['household_key', 'basket_id', 'day']).agg({
    'product_id': lambda x: list(x),
    'quantity': lambda x: x.tolist(),
    'store_id': 'first',
    'retail_disc': 'mean',
    'trans_time': 'first',
    'week_no': 'first',
    'coupon_disc': 'mean',
    'coupon_match_disc': 'mean',
    'sales_value_eu': lambda x: list(x),
    'loyalty_eu': lambda x: list(x),
    'non_loyalty_eu': lambda x: list(x),
    'transaction_date': lambda x: list(x)
}).reset_index()

basket.sort_values(by='day')


,household_key,basket_id,day,product_id,quantity,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,sales_value_eu,loyalty_eu,non_loyalty_eu,transaction_date
263587,2375,26984851516,1,"[826249, 1043142, 1085983, 1102651, 6423775, 9...","[2, 1, 1, 1, 1, 1]",364,-0.543333,1642,1,0.0,0.0,"[2.12, 1.68, 3.2, 2.02, 2.14, 2.14]","[1.38, 2.41, 3.63, 2.02, 2.99, 2.99]","[1.06, 1.68, 3.2, 2.02, 2.14, 2.14]","[2021-01-01, 2021-01-01, 2021-01-01, 2021-01-0..."
274742,2483,26992197681,1,[3843566],[2],2602,-0.980000,1823,1,0.0,0.0,[5.35],[3.2],[2.68],[2021-01-01]
263586,2375,26984851472,1,"[1004906, 1033142, 1036325, 1082185, 8160430]","[1, 1, 1, 1, 1]",364,-0.258000,1631,1,0.0,0.0,"[1.49, 0.88, 1.06, 1.29, 1.61]","[2.13, 0.88, 1.38, 1.29, 2.02]","[1.49, 0.88, 1.06, 1.29, 1.6]","[2021-01-01, 2021-01-01, 2021-01-01, 2021-01-0..."
141198,1287,26985336468,1,[5978648],[0],304,0.000000,1351,1,0.0,0.0,[0.0],[nan],[nan],[2021-01-01]
211515,1916,26996955143,1,"[826784, 855421, 885290, 918199, 920112, 92273...","[1, 1, 1, 2, 2, 1, 1, 1, 2, 5, 1, 1, 1, 2, 1, ...",446,-0.511739,2355,1,0.0,0.0,"[1.06, 3.84, 2.56, 2.25, 4.28, 2.66, 1.14, 5.3...","[1.59, 3.84, 2.56, 1.12, 2.66, 2.66, 2.34, 7.0...","[1.06, 3.84, 2.56, 1.12, 2.14, 2.66, 1.14, 5.3...","[2021-01-01, 2021-01-01, 2021-01-01, 2021-01-0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176607,1592,42289438534,711,[6534178],[12610],330,-1.260000,1737,102,0.0,0.0,[31.02],[0.0],[0.0],[2022-12-12]
223371,2019,42289792713,711,"[859702, 1065538, 17214937]","[1, 1, 1]",443,0.000000,2000,102,0.0,0.0,"[14.97, 17.11, 1.06]","[14.97, 17.11, 1.06]","[14.97, 17.11, 1.06]","[2022-12-12, 2022-12-12, 2022-12-12]"
252129,2270,42289457372,711,"[857849, 862139, 878676, 910473, 948650, 96155...","[1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1]",445,-0.546154,1846,102,0.0,0.0,"[0.6, 1.07, 2.68, 5.34, 3.54, 1.79, 2.84, 0.94...","[0.6, 0.66, 2.88, 5.34, 4.96, 2.13, 2.79, 1.53...","[0.6, 0.54, 2.68, 5.34, 3.54, 1.79, 1.42, 0.94...","[2022-12-12, 2022-12-12, 2022-12-12, 2022-12-1..."
127963,1158,42289631161,711,[916767],[2],289,0.000000,1714,102,0.0,0.0,[1.28],[0.64],[0.64],[2022-12-12]


Serialize a list to keep values as integers after exporting the dataset

In [39]:
# Apply serialization
basket['product_id'] = basket['product_id'].apply(functions.serialize_list)

## 4. Campaign

<a class='anchor' id='1'></a>
[Top &#129033;](#0) 

In [40]:
campaign.head(2)

,campaign,description,start_day,end_day,household_key,campaign_duration,start_date,end_date
0,26,TypeA,224,264,17,40,2021-08-12,2021-09-21
1,26,TypeA,224,264,27,40,2021-08-12,2021-09-21


Using *info* to explore datatypes and missing values

In [41]:
campaign.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7208 entries, 0 to 7207
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   campaign           7208 non-null   int64 
 1   description        7208 non-null   object
 2   start_day          7208 non-null   int64 
 3   end_day            7208 non-null   int64 
 4   household_key      7208 non-null   int64 
 5   campaign_duration  7208 non-null   int64 
 6   start_date         7208 non-null   object
 7   end_date           7208 non-null   object
dtypes: int64(5), object(3)
memory usage: 450.6+ KB


### 4.1 Dataset Transformation

Create a new dataframe where campaigns are grouped by household key

In [42]:
campaigns_per_household = campaign.groupby('household_key').agg({
    'campaign': lambda x: list(x),
    'description': lambda x: list(x),
    'start_day': lambda x: list(x),            
    'end_day': lambda x: list(x),               
    'campaign_duration': lambda x: list(x),     
    'start_date': lambda x: list(x),            
    'end_date': lambda x: list(x)               
}).reset_index()

campaigns_per_household.sort_values(by= 'household_key')

,household_key,campaign,description,start_day,end_day,campaign_duration,start_date,end_date
0,1,"[8, 13, 18, 29, 12, 22, 23, 20]","[TypeA, TypeA, TypeA, TypeB, TypeB, TypeB, Typ...","[412, 504, 587, 281, 477, 624, 646, 615]","[460, 551, 642, 334, 509, 656, 684, 685]","[48, 47, 55, 53, 32, 32, 38, 70]","[2022-02-16, 2022-05-19, 2022-08-10, 2021-10-0...","[2022-04-05, 2022-07-05, 2022-10-04, 2021-11-3..."
1,2,[18],[TypeA],[587],[642],[55],[2022-08-10],[2022-10-04]
2,3,"[8, 13, 14]","[TypeA, TypeA, TypeC]","[412, 504, 531]","[460, 551, 596]","[48, 47, 65]","[2022-02-16, 2022-05-19, 2022-06-15]","[2022-04-05, 2022-07-05, 2022-08-19]"
3,4,[8],[TypeA],[412],[460],[48],[2022-02-16],[2022-04-05]
4,6,"[8, 13, 18, 7]","[TypeA, TypeA, TypeA, TypeB]","[412, 504, 587, 398]","[460, 551, 642, 432]","[48, 47, 55, 34]","[2022-02-16, 2022-05-19, 2022-08-10, 2022-02-02]","[2022-04-05, 2022-07-05, 2022-10-04, 2022-03-08]"
...,...,...,...,...,...,...,...,...
1579,2496,"[8, 13, 18, 17, 14]","[TypeA, TypeA, TypeA, TypeB, TypeC]","[412, 504, 587, 575, 531]","[460, 551, 642, 607, 596]","[48, 47, 55, 32, 65]","[2022-02-16, 2022-05-19, 2022-08-10, 2022-07-2...","[2022-04-05, 2022-07-05, 2022-10-04, 2022-08-3..."
1580,2497,"[8, 13, 18]","[TypeA, TypeA, TypeA]","[412, 504, 587]","[460, 551, 642]","[48, 47, 55]","[2022-02-16, 2022-05-19, 2022-08-10]","[2022-04-05, 2022-07-05, 2022-10-04]"
1581,2498,"[8, 13, 18, 16, 17, 6]","[TypeA, TypeA, TypeA, TypeB, TypeB, TypeC]","[412, 504, 587, 561, 575, 393]","[460, 551, 642, 593, 607, 425]","[48, 47, 55, 32, 32, 32]","[2022-02-16, 2022-05-19, 2022-08-10, 2022-07-1...","[2022-04-05, 2022-07-05, 2022-10-04, 2022-08-1..."
1582,2499,"[8, 18]","[TypeA, TypeA]","[412, 587]","[460, 642]","[48, 55]","[2022-02-16, 2022-08-10]","[2022-04-05, 2022-10-04]"


## 5. Coupon

<a class='anchor' id='1'></a>
[Top &#129033;](#0) 

In [43]:
coupon.head(3)

,coupon_upc,product_id,household_key,day,campaign,trans_date,is_coupon_redeemed
0,10000089061,4,27160,NaN,NaN,NaN,False
1,10000089064,9,27754,1773.0,439.0,2022-03-15,True
2,10000089064,9,27754,321.0,446.0,2022-03-22,True


Using *info* and *isna* to explore datatypes and missing values

In [44]:
coupon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2218551 entries, 0 to 2218550
Data columns (total 7 columns):
 #   Column              Dtype  
---  ------              -----  
 0   coupon_upc          int64  
 1   product_id          int64  
 2   household_key       int64  
 3   day                 float64
 4   campaign            float64
 5   trans_date          object 
 6   is_coupon_redeemed  bool   
dtypes: bool(1), float64(2), int64(3), object(1)
memory usage: 103.7+ MB


In [45]:
coupon.isna().sum()

coupon_upc                0
product_id                0
household_key             0
day                   20189
campaign              20189
trans_date            20189
is_coupon_redeemed        0
dtype: int64

Since the values that are missing correpsond to coupons that were not redeemed, it makes sense that variables such as *day*, *campaign* and *trans_date* have missing values.

In [47]:
coupon[coupon['is_coupon_redeemed'] == False]

,coupon_upc,product_id,household_key,day,campaign,trans_date,is_coupon_redeemed
0,10000089061,4,27160,NaN,NaN,NaN,False
13,51800009050,28,28919,NaN,NaN,NaN,False
28,52100026076,28,28929,NaN,NaN,NaN,False
29,52100020075,1,28929,NaN,NaN,NaN,False
44,52100026076,28,29096,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...
2216219,52770031050,26,110028,NaN,NaN,NaN,False
2216220,52770031050,26,1111344,NaN,NaN,NaN,False
2216221,52770031050,26,1949053,NaN,NaN,NaN,False
2216222,53700067076,26,5125479,NaN,NaN,NaN,False


### 5.1 Dataset Transformation

In [48]:
coupons_per_household = coupon.groupby('household_key').agg({
    'coupon_upc': lambda x: list(x),
    'product_id': lambda x: list(x),
    'day': lambda x: list(x),            
    'campaign': lambda x: list(x),               
    'trans_date': lambda x: list(x),     
    'is_coupon_redeemed': lambda x: list(x)                    
}).reset_index()

coupons_per_household.sort_values(by= 'household_key')

,household_key,coupon_upc,product_id,day,campaign,trans_date,is_coupon_redeemed
0,25671,"[10000085427, 10000085427, 10000085427, 100000...","[13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 1...","[2489.0, 2427.0, 2280.0, 2130.0, 2085.0, 1965....","[515.0, 505.0, 512.0, 528.0, 531.0, 523.0, 523...","[2022-05-30, 2022-05-20, 2022-05-27, 2022-06-1...","[True, True, True, True, True, True, True, Tru..."
1,26190,"[10000089133, 10000089133, 51111070150, 100000...","[13, 13, 26, 8]","[2147.0, 1965.0, 788.0, 222.0]","[510.0, 509.0, 257.0, 415.0]","[2022-05-25, 2022-05-24, 2021-09-14, 2022-02-19]","[True, True, True, True]"
2,26601,"[10000085428, 10000085428, 10000085428, 100000...","[13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 1...","[2451.0, 2124.0, 2007.0, 1995.0, 1990.0, 1081....","[513.0, 541.0, 512.0, 516.0, 545.0, 548.0, 525...","[2022-05-28, 2022-06-25, 2022-05-27, 2022-05-3...","[True, True, True, True, True, True, True, Tru..."
3,26738,"[51111037233, 51111037250, 10000085427, 100000...","[26, 30, 13, 13, 13, 13, 13, 13, 13, 13, 13, 1...","[788.0, 239.0, 2489.0, 2427.0, 2280.0, 2130.0,...","[257.0, 332.0, 515.0, 505.0, 512.0, 528.0, 531...","[2021-09-14, 2021-11-28, 2022-05-30, 2022-05-2...","[True, True, True, True, True, True, True, Tru..."
4,26941,"[10000085427, 10000085427, 10000085427, 100000...","[13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 1...","[2489.0, 2427.0, 2280.0, 2130.0, 2085.0, 1965....","[515.0, 505.0, 512.0, 528.0, 531.0, 523.0, 523...","[2022-05-30, 2022-05-20, 2022-05-27, 2022-06-1...","[True, True, True, True, True, True, True, Tru..."
...,...,...,...,...,...,...,...
44128,18056453,"[10000085475, 10000085475, 10000085475, 100000...","[18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 1...","[2488.0, 2462.0, 2451.0, 2421.0, 2378.0, 2351....","[621.0, 602.0, 591.0, 605.0, 610.0, 598.0, 642...","[2022-09-13, 2022-08-25, 2022-08-14, 2022-08-2...","[True, True, True, True, True, True, True, Tru..."
44129,18104791,"[10000085475, 10000085475, 10000085475, 100000...","[18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 1...","[2488.0, 2462.0, 2451.0, 2421.0, 2378.0, 2351....","[621.0, 602.0, 591.0, 605.0, 610.0, 598.0, 642...","[2022-09-13, 2022-08-25, 2022-08-14, 2022-08-2...","[True, True, True, True, True, True, True, Tru..."
44130,18105264,"[10000085475, 10000085475, 10000085475, 100000...","[18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 1...","[2488.0, 2462.0, 2451.0, 2421.0, 2378.0, 2351....","[621.0, 602.0, 591.0, 605.0, 610.0, 598.0, 642...","[2022-09-13, 2022-08-25, 2022-08-14, 2022-08-2...","[True, True, True, True, True, True, True, Tru..."
44131,18122531,"[54589399250, 54589399250, 54589399250, 545893...","[14, 14, 14, 14, 27, 27]","[2489.0, 636.0, 588.0, nan, nan, nan]","[558.0, 533.0, 591.0, nan, nan, nan]","[2022-07-12, 2022-06-17, 2022-08-14, nan, nan,...","[True, True, True, False, False, False]"


## 6. Product

<a class='anchor' id='1'></a>
[Top &#129033;](#0) 

In [49]:
prod.head(3)

,product_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,curr_size_of_product_value,curr_size_of_product_units
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB,22.0,LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,,NaN,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,,NaN,


Using *info* to explore datatypes and missing values

In [50]:
prod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92353 entries, 0 to 92352
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  92353 non-null  int64  
 1   manufacturer                92353 non-null  int64  
 2   department                  92353 non-null  object 
 3   brand                       92353 non-null  object 
 4   commodity_desc              92353 non-null  object 
 5   sub_commodity_desc          92353 non-null  object 
 6   curr_size_of_product        92353 non-null  object 
 7   curr_size_of_product_value  61079 non-null  float64
 8   curr_size_of_product_units  92353 non-null  object 
dtypes: float64(1), int64(2), object(6)
memory usage: 6.3+ MB


### 6.1 Feature Engineering

Replace brand by a boolean

In [51]:
#prod['brand'].unique()
prod['brand_bin'] = prod['brand'].replace({
    'National': True,
    'Private': False
})

In [52]:
prod[prod['commodity_desc'] == ' ']

,product_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,curr_size_of_product_value,curr_size_of_product_units,brand_bin
52189,5126087,1,,National,,,,NaN,,True
52190,5126088,1,,National,,,,NaN,,True
52191,5126106,1,,National,,,,NaN,,True
52192,5126107,1,,National,,,,NaN,,True
55048,5977100,1,,National,,,,NaN,,True
55054,5978648,1,,National,,,,NaN,,True
55055,5978649,1,,National,,,,NaN,,True
55056,5978650,1,,National,,,,NaN,,True
55057,5978656,1,,National,,,,NaN,,True
55058,5978657,1,,National,,,,NaN,,True


drop products w/o commodity & sub_commodity

In [53]:
prod = prod[prod['commodity_desc'] != ' '].reset_index(drop=True)

Since the number of departments is very extensive, some will be merged together.

In [54]:
prod['department'].unique()

array(['GROCERY', 'MISC. TRANS.', 'PASTRY', 'DRUG GM', 'MEAT-PCKGD',
       'SEAFOOD-PCKGD', 'PRODUCE', 'NUTRITION', 'DELI', 'COSMETICS',
       'MEAT', 'FLORAL', 'TRAVEL & LEISUR', 'SEAFOOD', 'MISC SALES TRAN',
       'SALAD BAR', 'KIOSK-GAS', 'ELECT &PLUMBING', 'GRO BAKERY',
       'GM MERCH EXP', 'FROZEN GROCERY', 'COUP/STR & MFG', 'SPIRITS',
       'GARDEN CENTER', 'TOYS', 'CHARITABLE CONT', 'RESTAURANT', 'RX',
       'PROD-WHS SALES', 'MEAT-WHSE', 'DAIRY DELI', 'CHEF SHOPPE', 'HBC',
       'DELI/SNACK BAR', 'PORK', 'AUTOMOTIVE', 'VIDEO RENTAL',
       'CNTRL/STORE SUP', 'HOUSEWARES', 'POSTAL CENTER', 'PHOTO', 'VIDEO',
       'PHARMACY SUPPLY'], dtype=object)

In [55]:
replacement_mapping = {
    'MEAT': 'Meat',
    'MEAT-PCKGD': 'Meat',
    'MEAT-WHSE': 'Meat',
    'PORK': 'Meat',
    'SEAFOOD': 'Seafood',
    'SEAFOOD-PCKGD': 'Seafood',
    'FROZEN GROCERY': 'Groceries',
    'PRODUCE': 'Groceries',
    'GROCERY': 'Groceries',
    'NUTRITION': 'Groceries',
    'GRO BAKERY': 'Bakery',
    'PASTRY': 'Bakery',
    'DELI': 'Delicacies',
    'DAIRY DELI': 'Delicacies',
    'DELI/SNACK BAR': 'DELI/SNACK BAR',
    'PHOTO': 'Photo/Video',
    'VIDEO': 'Photo/Video'
}

prod['department'] = prod['department'].replace(replacement_mapping)


## 7. Causal

<a class='anchor' id='1'></a>
[Top &#129033;](#0) 

In [57]:
causal.head(3)

,product_id,store_id,week_no,display,mailer,display_description,mailer_description
0,770829,432,84,9,0,Secondary Location Display,Not an Ad
1,770829,433,13,5,H,Rear End Cap,Wrap front feature
2,770829,433,29,0,A,Not On Display,Interior page feature


Using *info* to explore datatypes and missing values

In [58]:
causal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36786524 entries, 0 to 36786523
Data columns (total 7 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   product_id           int64 
 1   store_id             int64 
 2   week_no              int64 
 3   display              object
 4   mailer               object
 5   display_description  object
 6   mailer_description   object
dtypes: int64(3), object(4)
memory usage: 1.9+ GB


### 7.1 Dataset Transformation

Grouping data by product

In [59]:
causal_per_prod_id = causal.groupby('product_id').agg({
    'store_id': lambda x: list(x),
    'week_no': lambda x: list(x),
    'display': lambda x: list(x),            
    'mailer': lambda x: list(x),               
    'display_description': lambda x: list(x),     
    'mailer_description': lambda x: list(x)                    
}).reset_index()

causal_per_prod_id.sort_values(by= 'product_id')

,product_id,store_id,week_no,display,mailer,display_description,mailer_description
0,26190,"[286, 288, 289, 292, 293, 295, 296, 297, 298, ...","[70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 7...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 5, 0, ...","[A, A, A, A, A, A, A, A, A, A, A, A, A, A, A, ...","[Not On Display, Not On Display, Not On Displa...","[Interior page feature, Interior page feature,..."
1,26355,"[286, 286, 286, 286, 286, 286, 286, 286, 286, ...","[35, 37, 38, 41, 42, 43, 85, 87, 88, 89, 90, 9...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Rear End Cap, Rear End Cap, Rear End Cap, Rea...","[Not an Ad, Not an Ad, Not an Ad, Not an Ad, N..."
2,26426,"[286, 286, 286, 286, 286, 288, 288, 288, 288, ...","[30, 31, 35, 51, 55, 30, 31, 32, 51, 55, 30, 3...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[A, A, A, A, A, A, A, A, A, A, A, A, A, A, A, ...","[Not On Display, Not On Display, Not On Displa...","[Interior page feature, Interior page feature,..."
3,26540,"[289, 313, 315, 316, 316, 316, 316, 316, 316, ...","[54, 56, 25, 52, 53, 54, 55, 56, 57, 59, 61, 6...","[3, 5, 4, 6, 6, 6, 6, 9, 9, 6, 6, 6, 6, 6, 6, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Front End Cap, Rear End Cap, Mid-Aisle End Ca...","[Not an Ad, Not an Ad, Not an Ad, Not an Ad, N..."
4,26601,"[286, 286, 286, 288, 288, 288, 288, 288, 288, ...","[43, 71, 78, 13, 43, 61, 63, 78, 95, 97, 13, 2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[A, A, A, H, A, A, A, A, D, A, H, A, A, A, A, ...","[Not On Display, Not On Display, Not On Displa...","[Interior page feature, Interior page feature,..."
...,...,...,...,...,...,...,...
68372,18203132,[356],[101],[1],[0],[Store Front],[Not an Ad]
68373,18203921,"[341, 341, 424, 439, 439]","[100, 101, 101, 100, 101]","[1, 1, 1, 1, 1]","[0, 0, 0, 0, 0]","[Store Front, Store Front, Store Front, Store ...","[Not an Ad, Not an Ad, Not an Ad, Not an Ad, N..."
68374,18243471,[415],[101],[3],[0],[Front End Cap],[Not an Ad]
68375,18243476,[415],[101],[3],[0],[Front End Cap],[Not an Ad]


## 8. Export

<a class='anchor' id='1'></a>
[Top &#129033;](#0) 

In [61]:
path = '/Users/antoniooliveira/Downloads/NTT project/Gold'
#path = "C:/Users/aprataso/Downloads/final_data/Gold"

hh.to_csv(f'{path}/hh_treated.csv', index=False)
basket.to_csv(f'{path}/basket_treated.csv', index=False)
campaigns_per_household.to_csv(f'{path}/campaigns_treated.csv', index=False)
coupons_per_household.to_csv(f'{path}/coupon_treated.csv', index=False)
prod.to_csv(f'{path}/prod_treated.csv', index=False)
causal_per_prod_id.to_csv(f'{path}/causal_treated.csv', index=False)